<a href="https://colab.research.google.com/github/mosalov/personal_trajectories/blob/main/stud_(20_11_2021)_ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Траектории студентов

Загрузка библиотек

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import math

Загрузка файла

In [2]:
def load_data(filename):
    df = pd.read_excel(filename)
    return df

In [3]:
filename = '/content/drive/MyDrive/Colab Notebooks/edu_results.xlsx'

data = load_data(filename)

Удаляем точки в данных и пробелы. Приводим все значения к одному регистру

In [27]:
new_data = data[data['sem'] == 1].copy()
del new_data['sem']
print(new_data.shape)

(106921, 8)


In [28]:
# Нужно использовать не просто replace, а str.replace!
# Удалить внутренности скобок. Скобки приходится экранировать с помощью \, иначе они считаются частью регулярного выражения.
new_data['subj'] = new_data['subj'].str.replace('\(.*\)', '', regex=True)

In [29]:
new_data['subj'] = new_data['subj'].str.replace('МДК', '', regex=True)

In [30]:
# Удалить цифры
new_data['subj'] = new_data['subj'].str.replace('\d+', '', regex=True)

In [31]:
# Удаление точек
new_data['subj'] = new_data['subj'].str.replace('\.', '', regex=True)

In [32]:
# Удаление пробелов в начале и конце каждого значения
new_data['subj'] = new_data['subj'].str.strip()

In [33]:
# Преведение к нижнем регистру
# Тут тоже через str.lower!
# Добавил присвоение
new_data['subj'] = new_data['subj'].str.lower()

In [34]:
del new_data['filial']

In [35]:
del new_data['sttext']

In [36]:
del new_data['formobtext']

In [37]:
del new_data['grouptext']

In [38]:
del new_data['yearin']

Преобразование предметов в колонки для анализа оценок по предметам. Если предмета нет, то -1

In [39]:
new_data['mark'] = new_data['mark'].astype('str').str.replace('0', '1', regex=True).astype('int')

In [40]:
# Замена оценок, которые больше 5, на 1
new_data[new_data['mark'] > 5] = 1

In [41]:
# One-hot-encoding столбца subj
new_data = new_data.join(pd.get_dummies(new_data.subj, prefix='subj'))

In [42]:
# Размер таблицы после one-hot-encoding
new_data.shape

(106921, 262)

In [43]:
# Умножение Названий предметов на оценку

In [44]:
cols = new_data.columns.str.startswith('subj_')
print(cols)
print(len(cols))

[False False False  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  T

In [45]:
from tqdm import tqdm

for c in tqdm(new_data.columns[cols]):
    new_data[c] = new_data[c].astype('int').multiply(new_data['mark'], axis=0)

100%|██████████| 259/259 [00:01<00:00, 163.01it/s]


In [49]:
del new_data['mark']

In [50]:
new_data.shape

(106921, 261)

In [51]:
# Для группировки данных по студенту/семестру должно сработать это:
new_data1 = new_data.groupby('hash').sum()

In [52]:
new_data1

,subj_административное право,subj_академическая живопись,subj_академический рисунок,subj_академический рисунок и скульптура,subj_актуальные вопросы финансов,subj_актуальные проблемы финансов,subj_алгебра логики и дискретный анализ,subj_анализ статистической информации с помощью пакетов прикладных программ,subj_аналитическая геометрия,subj_анатомия и физиология человека,subj_анатомия центральной нервной системы,subj_астрономия,subj_безопасность жизнедеятельности,subj_биология,subj_бухгалтерский учет и аудит,subj_бюджетирование и корпоративное финансовое планирование,subj_бюджетная система рф,subj_введение в коммуникационные специальности,subj_введение в профессию,"subj_введение в профессию ""управление качеством""",subj_введение в специальность,subj_введение в специальную технику,subj_введение в языкознание,subj_всеобщая история,subj_выполнение работ по должности служащего делопроизводитель,"subj_выполнение работ по одной или нескольким профессиям рабочих, должностям служащих",subj_выполнение работ по профессии «кассир»,subj_высшая математика,subj_география,subj_государственное регулирование экономики,subj_гражданский процесс,subj_девиантология,subj_деловая этика,subj_деловой иностранный язык,subj_деловые коммуникации,subj_дизайн и монументально-декоративное искусство в формировании среды,subj_дизайн проектирования,subj_дизайн-проектирование,subj_документационное обеспечение управления,subj_документоведение,...,subj_теория организации и организационное поведение,subj_техническая механика,subj_техническое черчение,subj_технологии и среды программирования,subj_технология ведения домашнего хозяйства,subj_технология создания и обработки цифровой информации,subj_уп учебная практика,"subj_управление качеством проекта, проектная документация",subj_управленческая экономика,subj_учебная практика,subj_учебная: ознакомительная практика,subj_учебная: практика по получению первичных профессиональных умений и навыков,"subj_учебная: практика по получению первичных профессиональных умений и навыков, в том числе первичных умений и навыков научно-исследовательской деятельности",subj_учебная: творческая практика,subj_физика,subj_физическая культура,subj_физкультура,subj_философия,subj_философия и методология социальных наук,subj_философские проблемы науки и техники,"subj_финансы, денежное обращение и кредит",subj_формирование клиентской базы,subj_химия,subj_экологические основы природопользования,subj_экология,subj_экономика,subj_экономика организации,subj_экономика предприятия,subj_экономика предприятия и организация производства,subj_экономико-математические методы моделирования,subj_экономическая социология в россии,subj_экономическая теория,subj_элективные курсы по физической культуре,subj_элективные курсы по физической культуре и спорту,subj_электротехника и электроника,subj_электротехника и электронная техника,subj_элементы высшей математики,subj_этика делового общения,subj_этика деловых отношений,subj_языки высокого уровня
hash,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0x000DE9673F941872AFFC6EF39B3B0EEAC868BA3B,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,5,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,4
0x00153956450F17DDD7C0610CF52E1476DE4E65E8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,5,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0x0019D4D822B0D9A3DD3336438904749E38147903,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,5,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0x001D374A732EE3FE32F65550A2D7E970C5EB7AE1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
0x0026AD5250A4C255F6687FBE6292D8EBE10915E7,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [54]:
new_data1.to_csv('/content/drive/MyDrive/Colab Notebooks/prepared_data_sem_1.csv', index=False)